In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import datetime
from sklearn.multioutput import MultiOutputRegressor
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import json
train_data = pd.read_csv("train.csv",converters={'POLYLINE':lambda x: json.loads(x)})#(320, 9) 將POLYINE從str->list
test_data = pd.read_csv("test.csv",converters={'POLYLINE':lambda x: json.loads(x)})#(320, 9)

In [3]:
all_data = train_data.append(test_data)
all_data = all_data[all_data['POLYLINE'].str.len() > 0] #將POLYLIME為空值的數據過濾掉
#-----------CALL_TYPE處理 & ORIGIN_CALL處理-----------#
all_data = all_data.fillna(0)#填缺
all_data.loc[all_data['CALL_TYPE'] != 0, 'CALL_TYPE' ] = 1
all_data.loc[all_data['ORIGIN_CALL'] != 0, 'ORIGIN_CALL' ] = 1
#---------------------時間-------------------#
all_data["TIMESTAMP"] = [float(time) for time in all_data["TIMESTAMP"]]
all_data["data_time"] = [datetime.datetime.fromtimestamp(time, datetime.timezone.utc) for time in all_data["TIMESTAMP"]]
all_data['data_time']
all_data["Year"] = all_data["data_time"].dt.year
all_data["Month"] = all_data["data_time"].dt.month
all_data["Day"] = all_data["data_time"].dt.day
all_data["Hour"] = all_data["data_time"].dt.hour
all_data["Minute"] = all_data["data_time"].dt.minute
all_data["Weekday"] = all_data["data_time"].dt.weekday
#-------all_data-------------其餘處理-----------------#
all_data['CALL_TYPE'] = all_data['CALL_TYPE'].map(dict(A=1, B=2, C=3))#labelEncode
all_data["POLYLINE_target_x"] = all_data["POLYLINE"].apply(lambda x: x[len(x) - 1][0])#將x target提出來
all_data["POLYLINE_target_y"] = all_data["POLYLINE"].apply(lambda x: x[len(x) - 1][1])#將y target提出來 
#----------計程車車程時間------------------#
all_data["Travel_time"] = all_data["POLYLINE"].apply(lambda x: len(x))#將x target提出來
#----------------將all_data拆分成原本的train_data和test_data--------------------#
train_data_than_len0 = len(train_data[train_data['POLYLINE'].str.len() > 0])
test_data_than_len0 = len(test_data[test_data['POLYLINE'].str.len() > 0])
train_data = all_data[0:train_data_than_len0]
test_data = all_data[train_data_than_len0:(train_data_than_len0 + test_data_than_len0)]
#----------------將train和test拆分成 < 11的 >= 11和的---------------------------#
train_data_than11 = train_data[train_data['POLYLINE'].str.len() >= 3]
train_data_less11 = train_data[train_data['POLYLINE'].str.len() < 3]
test_data_than10 = test_data[test_data['POLYLINE'].str.len() >= 2]
test_data_less10 = test_data[test_data['POLYLINE'].str.len() < 2]

In [4]:
for i in range(1,2):
    train_data_than11["POLYLINE_first_x" + str(i)] = train_data_than11["POLYLINE"].apply(lambda x: x[i - 1][0])#將label提出來
    train_data_than11["POLYLINE_first_y" + str(i)] = train_data_than11["POLYLINE"].apply(lambda x: x[i - 1][1])#將label提出來
    test_data_than10["POLYLINE_first_x" + str(i)] = test_data_than10["POLYLINE"].apply(lambda x: x[i - 1][0])#將label提出來
    test_data_than10["POLYLINE_first_y" + str(i)] = test_data_than10["POLYLINE"].apply(lambda x: x[i - 1][1])#將label提出來
for i in range(1,0,-1):
    train_data_than11["POLYLINE_final_x" + str(2-i)] = train_data_than11["POLYLINE"].apply(lambda x: x[len(x) - 1 - i][0])#將label提出來
    train_data_than11["POLYLINE_final_y" + str(2-i)] = train_data_than11["POLYLINE"].apply(lambda x: x[len(x) - 1 - i][1])#將label提出來
    test_data_than10["POLYLINE_final_x" + str(2-i)] = test_data_than10["POLYLINE"].apply(lambda x: x[len(x) - i][0])#將label提出來
    test_data_than10["POLYLINE_final_y" + str(2-i)] = test_data_than10["POLYLINE"].apply(lambda x: x[len(x) - i][1])#將label提出來
    
train_data_than11["x_vector"] = train_data_than11["POLYLINE_final_x1"] - train_data_than11["POLYLINE_first_x1"]
train_data_than11["y_vector"] = train_data_than11["POLYLINE_final_y1"] - train_data_than11["POLYLINE_first_y1"]
train_data_than11 = train_data_than11.query("x_vector <= 0.2 & x_vector >= -0.2 & y_vector <= 0.2 & y_vector >= -0.2")
test_data_than10["x_vector"] = test_data_than10["POLYLINE_final_x1"] - test_data_than10["POLYLINE_first_x1"]
test_data_than10["y_vector"] = test_data_than10["POLYLINE_final_y1"] - test_data_than10["POLYLINE_first_y1"]

C:\Users\love1\AppData\Local\Temp/ipykernel_5432/2974239374.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_than11["POLYLINE_first_x" + str(i)] = train_data_than11["POLYLINE"].apply(lambda x: x[i - 1][0])#將label提出來
C:\Users\love1\AppData\Local\Temp/ipykernel_5432/2974239374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_than11["POLYLINE_first_y" + str(i)] = train_data_than11["POLYLINE"].apply(lambda x: x[i - 1][1])#將label提出來
C:\Users\love1\AppData\Local\Temp/ipykernel_5432/297

In [ ]:
from sklearn.model_selection import train_test_split
train_data_than11, valid_data = train_test_split(train_data_than11, random_state=123, train_size=0.9)

In [17]:
'''mapping_1st = pd.DataFrame({
    
    "lon":train_data_than11["POLYLINE_first_x1"].head(100000).values,
    "lat":train_data_than11["POLYLINE_first_y1"].head(100000).values
})'''
data_last = pd.DataFrame({
    
    "lon":train_data_than11["POLYLINE_target_x"].head(100000).values,
    "lat":train_data_than11["POLYLINE_target_y"].head(100000).values
})
#all_xy = pd.concat([mapping_last])
from sklearn.cluster import KMeans
kmean_model = KMeans(n_clusters=3200,init='k-means++').fit(data_last)
#inertias = [model.cluster_centers_ for model in kmeans_list]

#kmeans_list = [KMeans(n_clusters=3200).fit(data_last)]
#inertias = [model.cluster_centers_ for model in kmeans_list]
#a = pd.DataFrame(inertias[0])

In [6]:
predict_final = ['CALL_TYPE','x_vector','y_vector','POLYLINE_first_x1','ORIGIN_CALL','ORIGIN_STAND','POLYLINE_first_y1']

from xgboost import XGBRegressor
XGBR_x = XGBRegressor()
XGBR_x.fit(train_data_than11[predict_final],train_data_than11['POLYLINE_target_x'])
XGBR_y = XGBRegressor()
XGBR_y.fit(train_data_than11[predict_final],train_data_than11['POLYLINE_target_y'])

'''from xgboost import XGBRegressor    
XGBR = MultiOutputRegressor(XGBRegressor())
XGBR = XGBR.fit(train_data_than11[predict_final], train_data_than11[['POLYLINE_target_x','POLYLINE_target_y']])
XGBR_x.fit(train_data_than11[predict_final],train_data_than11['POLYLINE_target_x'])
XGBR_y.fit(train_data_than11[predict_final],train_data_than11['POLYLINE_target_y'])'''

"from xgboost import XGBRegressor    \nXGBR = MultiOutputRegressor(XGBRegressor())\nXGBR = XGBR.fit(train_data_than11[predict_final], train_data_than11[['POLYLINE_target_x','POLYLINE_target_y']])\nXGBR_x.fit(train_data_than11[predict_final],train_data_than11['POLYLINE_target_x'])\nXGBR_y.fit(train_data_than11[predict_final],train_data_than11['POLYLINE_target_y'])"

In [15]:
pred_XGB_x = XGBR_x.predict(test_data_than10[predict_final])
test_data_than10['LONGITUDE'] = pred_XGB_x
pred_XGB_y = XGBR_y.predict(test_data_than10[predict_final])
test_data_than10['LATITUDE'] = pred_XGB_y

test_data_less10['LONGITUDE'] = test_data_less10['POLYLINE_target_x']
test_data_less10['LATITUDE'] = test_data_less10['POLYLINE_target_y']

test_result = pd.concat([test_data_than10, test_data_less10])
test_result = test_result.sort_index()

C:\Users\love1\AppData\Local\Temp/ipykernel_5432/2565689721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_than10['LONGITUDE'] = pred_XGB_x
C:\Users\love1\AppData\Local\Temp/ipykernel_5432/2565689721.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_than10['LATITUDE'] = pred_XGB_y
C:\Users\love1\AppData\Local\Temp/ipykernel_5432/2565689721.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [ ]:
pred = kmean_model.predict(test_result[['LONGITUDE','LATITUDE']])
center = kmean_model.cluster_centers_
pred_center = []
for p in pred:
    pred_center.append(center[p])
result = pd.DataFrame(pred_center)

submission = pd.DataFrame({
    'TRIP_ID':test_result['TRIP_ID'],
    'LATITUDE':result[1],
    'LONGITUDE':result[0]
                         })
submission.to_csv('submission.csv', index=False)

In [ ]:
submission = pd.DataFrame({
    'TRIP_ID':test_result['TRIP_ID'],
    'LATITUDE':test_result['LATITUDE'],
    'LONGITUDE':test_result['LONGITUDE']
                         })
submission.to_csv('submission.csv', index=False)

In [ ]:
print(train_data_less11.shape)
print(train_data_than11.shape)
print(test_data_less10.shape)
print(test_data_than10.shape)

In [ ]:
##-----------------------以下程式碼為驗證階段---------------------------##
from sklearn.metrics import mean_squared_error 

pred_valid_x = XGBR_x.predict(valid_data[predict_final])
valid_data['LONGITUDE'] = pred_valid_x
pred_valid_y = XGBR_y.predict(valid_data[predict_final])
valid_data['LATITUDE'] = pred_valid_y

print('驗證集分數: ',mean_squared_error(valid_data['POLYLINE_target_x'],valid_data['LONGITUDE']) + \
      mean_squared_error(valid_data['POLYLINE_target_y'],valid_data['LATITUDE']))

In [ ]:
valid_data.info()

In [18]:
import folium
a = pd.DataFrame(kmean_model.cluster_centers_)

mapping_keman = pd.DataFrame({
    
    "lon":a[0].values,
    "lat":a[1].values
})
mapping_final = pd.DataFrame({
    
    "lon":test_data_than10["POLYLINE_final_x1"].values,
    "lat":test_data_than10["POLYLINE_final_y1"].values
})

mapping_pred = pd.DataFrame({
    
    "lon":test_data_than10["LONGITUDE"].head(5000).values,
    "lat":test_data_than10["LATITUDE"].head(5000).values
})
'''mapping_truth = pd.DataFrame({
    
    "lon":valid_data["POLYLINE_target_x"].head(5000).values,
    "lat":valid_data["POLYLINE_target_y"].head(5000).values
})'''

por_map = folium.Map(location=[41.141412,-8.590324], tiles='Stamen Terrain', zoom_start=11)

for i, r in mapping_keman.iterrows():
    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.2, color="yellow").add_to(por_map)    
    
for i, r in mapping_final.iterrows():
    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.2, color="purple").add_to(por_map)    

for i, r in mapping_pred.iterrows():
    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.2, color="blue").add_to(por_map)    

#for i, r in mapping_truth.iterrows():
#    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.2, color="red").add_to(por_map)   
    
por_map

In [ ]:
import folium

mapping_pred = pd.DataFrame({
    
    "lon":valid_data["LONGITUDE"].head(1000).values,
    "lat":valid_data["LATITUDE"].head(1000).values
})
mapping_truth = pd.DataFrame({
    
    "lon":valid_data["POLYLINE_target_x"].head(1000).values,
    "lat":valid_data["POLYLINE_target_y"].head(1000).values
})

mapping_final = pd.DataFrame({
    
    "lon":valid_data["POLYLINE_final_x1"].head(1000).values,
    "lat":valid_data["POLYLINE_final_y1"].head(1000).values
})

por_map = folium.Map(location=[41.141412,-8.590324], tiles='Stamen Terrain', zoom_start=11)

for i, r in mapping_pred.iterrows():
    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.2, color="blue").add_to(por_map)

for i, r in mapping_truth.iterrows():
    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.2, color="red").add_to(por_map)    

for i, r in mapping_final.iterrows():
    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.2, color="yellow").add_to(por_map)    
    
#for i,r in meta_data.iterrows():
#    folium.CircleMarker(location=[r["Latitude"],r["Longitude"]], radius=2, color="yellow").add_to(por_map)    
    
por_map

In [ ]:
from xgboost import plot_importance
plot_importance(XGBR_x)
plot_importance(XGBR_y)